In [1]:
#Data Preprocessing And Cleaning
import re
import pandas as pd
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

#Step 1: Environment Setup
!pip install pandas numpy openpyxl keybert transformers torch scikit-learn


# Loading the Merged Comment data
comments_df = pd.read_csv("G:/Other computers/My Laptop (1)/Pelajaran UiTM/Classes and Lectures (Semester 5)/FINAL YEAR PROJECT/Developments/Simplified Data/Merged Files/Merged Comments V3.csv")

# Loading the Video with Transcription data
videos_df = pd.read_excel("G:/Other computers/My Laptop (1)/Pelajaran UiTM/Classes and Lectures (Semester 5)/FINAL YEAR PROJECT/Developments/Simplified Data/Transcriptions/Video with Transcription.xlsx")

#Inspecting Data
comments_df.head()
videos_df.head()

# Step 2: Handling Missing Values Section
# Check missing values in comments
print(comments_df.isnull().sum())

# Remove missing values or fill with placeholder text
comments_df.dropna(subset=['comments'], inplace=True)

# Alternatively, to fill:
# comments_df['comments'].fillna("No comment provided", inplace=True)

# Check missing values in videos
print(videos_df.isnull().sum())

videos_df.dropna(subset=['transcription'], inplace=True)

# Step 3: Removing duplicates Section
# Remove duplicates from comments
comments_df.drop_duplicates(subset=['comments'], inplace=True)

# Remove duplicates from videos
videos_df.drop_duplicates(subset=['transcription'], inplace=True)

# Step4: Text Normalization & Cleaning
def clean_text(text):
    text = str(text).lower()  # convert text to lowercase
    text = re.sub(r'http\S+', '', text)  # remove URLs
    text = re.sub(r'[^\w\s]', '', text)  # remove punctuation
    text = re.sub(r'\d+', '', text)  # remove numbers
    text = re.sub(r'\s+', ' ', text).strip()  # normalize whitespace
    return text

# Apply cleaning to comments and transcriptions
comments_df['clean_comments'] = comments_df['comments'].apply(clean_text)
videos_df['clean_transcriptions'] = videos_df['transcription'].apply(clean_text)

#Step 5: Tokenization
# Tokenize the cleaned comments and transcriptions
comments_df['tokens'] = comments_df['clean_comments'].apply(word_tokenize)
videos_df['tokens'] = videos_df['clean_transcriptions'].apply(word_tokenize)

# Step 6: Stopword Removal
stop_words = set(stopwords.words('english'))

# Define function for removing stopwords
def remove_stopwords(tokens):
    return [word for word in tokens if word not in stop_words]

comments_df['tokens_no_stopwords'] = comments_df['tokens'].apply(remove_stopwords)
videos_df['tokens_no_stopwords'] = videos_df['tokens'].apply(remove_stopwords)

#Step 7: Lemmatization
lemmatizer = WordNetLemmatizer()

def lemmatize_tokens(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens]

comments_df['lemmatized'] = comments_df['tokens_no_stopwords'].apply(lemmatize_tokens)
videos_df['lemmatized'] = videos_df['tokens_no_stopwords'].apply(lemmatize_tokens)

#Step 8: Preparing for Keyword Extraction (KeyBERT)
comments_df['final_text'] = comments_df['lemmatized'].apply(lambda x: ' '.join(x))
videos_df['final_text'] = videos_df['lemmatized'].apply(lambda x: ' '.join(x))

#Step 9: Save Preprocessed Data
comments_df.to_csv('clean_comments.csv', index=False)
videos_df.to_csv('clean_transcriptions.csv', index=False)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...


Vid             135
Cid               0
RepliesToId    5982
Comments          0
uniqueId          4
videoWebUrl       0
dtype: int64


KeyError: ['comments']

In [31]:
#STEP 9
#Installing NLTK
#Pass this point is successful one

import nltk

# List of essential NLTK resources
resources = [
    ('punkt', 'tokenizers/punkt'),
    ('stopwords', 'corpora/stopwords'),
    ('wordnet', 'corpora/wordnet'),
    ('omw-1.4', 'corpora/omw-1.4')  # Optional, for extended lemmatization support
]

for resource_name, resource_path in resources:
    try:
        nltk.data.find(resource_path)
        print(f"✅ '{resource_name}' already exists.")
    except LookupError:
        print(f"⏳ Downloading '{resource_name}'...")
        nltk.download(resource_name)

# Verification
print("\n🔍 Verifying downloads...")
for resource_name, resource_path in resources:
    try:
        nltk.data.find(resource_path)
        print(f"✅ '{resource_name}' successfully downloaded and verified.")
    except LookupError:
        print(f"❌ '{resource_name}' download failed. Please try manually.")


✅ 'punkt' already exists.
✅ 'stopwords' already exists.
⏳ Downloading 'wordnet'...
⏳ Downloading 'omw-1.4'...

🔍 Verifying downloads...
✅ 'punkt' successfully downloaded and verified.
✅ 'stopwords' successfully downloaded and verified.
❌ 'wordnet' download failed. Please try manually.
❌ 'omw-1.4' download failed. Please try manually.


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [13]:
# STEPS 10
# KeyBERT Implementation - Extract keywords

!pip install emoji
import tkinter as tk
from tkinter import filedialog, ttk, messagebox, scrolledtext
import threading
import os
import re
import sys
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from keybert import KeyBERT
import time
import traceback
import emoji # <-- Import the emoji library

# Download NLTK resources (Consider doing this once outside the main script execution if possible)
# Ensure NLTK data path is correctly set up if needed
try:
    nltk.data.find('tokenizers/punkt')
except nltk.downloader.DownloadError:
    nltk.download('punkt', quiet=True)
try:
    nltk.data.find('corpora/stopwords')
except nltk.downloader.DownloadError:
    nltk.download('stopwords', quiet=True)
try:
    nltk.data.find('corpora/wordnet')
except nltk.downloader.DownloadError:
    nltk.download('wordnet', quiet=True)
try:
    nltk.data.find('corpora/omw-1.4')
except nltk.downloader.DownloadError:
    nltk.download('omw-1.4', quiet=True)


class KeywordExtractionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Keyword Extraction Tool")
        self.root.geometry("800x600")
        self.root.resizable(True, True)

        # Initialize variables
        self.comments_file_path = tk.StringVar()
        self.videos_file_path = tk.StringVar()
        self.output_dir = tk.StringVar()
        self.output_dir.set(os.getcwd())
        self.progress_var = tk.DoubleVar()
        self.status_var = tk.StringVar()
        self.status_var.set("Ready")
        self.processing_thread = None
        self.stop_requested = False

        # Configure the main frame
        main_frame = ttk.Frame(root, padding="10")
        main_frame.pack(fill=tk.BOTH, expand=True)

        # Create the input file selection section
        self.create_file_selection_section(main_frame)

        # Create the settings section
        self.create_settings_section(main_frame)

        # Create the output section
        self.create_output_section(main_frame)

        # Create the log section
        self.create_log_section(main_frame)

        # Create the control buttons section
        self.create_control_buttons(main_frame)

        # Configure progress bar
        self.progress_bar = ttk.Progressbar(main_frame, variable=self.progress_var, length=780)
        self.progress_bar.pack(fill=tk.X, padx=10, pady=5)

        # Configure status bar
        status_bar = ttk.Label(main_frame, textvariable=self.status_var, relief=tk.SUNKEN, anchor=tk.W)
        status_bar.pack(fill=tk.X, padx=10, pady=5)

    def create_file_selection_section(self, parent):
        file_frame = ttk.LabelFrame(parent, text="Input Files", padding="10")
        file_frame.pack(fill=tk.X, padx=10, pady=5)

        # Comments file selector
        ttk.Label(file_frame, text="Comments File:").grid(row=0, column=0, sticky=tk.W, padx=5, pady=5)
        ttk.Entry(file_frame, textvariable=self.comments_file_path, width=60).grid(row=0, column=1, padx=5, pady=5)
        ttk.Button(file_frame, text="Browse...", command=self.browse_comments_file).grid(row=0, column=2, padx=5, pady=5)

        # Videos file selector
        ttk.Label(file_frame, text="Videos File:").grid(row=1, column=0, sticky=tk.W, padx=5, pady=5)
        ttk.Entry(file_frame, textvariable=self.videos_file_path, width=60).grid(row=1, column=1, padx=5, pady=5)
        ttk.Button(file_frame, text="Browse...", command=self.browse_videos_file).grid(row=1, column=2, padx=5, pady=5)

    def create_settings_section(self, parent):
        settings_frame = ttk.LabelFrame(parent, text="Processing Settings", padding="10")
        settings_frame.pack(fill=tk.X, padx=10, pady=5)

        # Comment column name
        ttk.Label(settings_frame, text="Comment Column:").grid(row=0, column=0, sticky=tk.W, padx=5, pady=5)
        self.comment_column_var = tk.StringVar(value="Comments") # Default value
        ttk.Entry(settings_frame, textvariable=self.comment_column_var, width=20).grid(row=0, column=1, padx=5, pady=5)

        # Transcription column name
        ttk.Label(settings_frame, text="Transcription Column:").grid(row=0, column=2, sticky=tk.W, padx=5, pady=5)
        self.transcription_column_var = tk.StringVar(value="transcription") # Default value
        ttk.Entry(settings_frame, textvariable=self.transcription_column_var, width=20).grid(row=0, column=3, padx=5, pady=5)

        # Top N keywords
        ttk.Label(settings_frame, text="Number of Keywords:").grid(row=1, column=0, sticky=tk.W, padx=5, pady=5)
        self.top_n_var = tk.IntVar(value=5)
        ttk.Spinbox(settings_frame, from_=1, to=20, textvariable=self.top_n_var, width=5).grid(row=1, column=1, padx=5, pady=5)

        # Batch size
        ttk.Label(settings_frame, text="Batch Size:").grid(row=1, column=2, sticky=tk.W, padx=5, pady=5)
        self.batch_size_var = tk.IntVar(value=100)
        ttk.Spinbox(settings_frame, from_=10, to=500, textvariable=self.batch_size_var, width=5).grid(row=1, column=3, padx=5, pady=5)

    def create_output_section(self, parent):
        output_frame = ttk.LabelFrame(parent, text="Output Settings", padding="10")
        output_frame.pack(fill=tk.X, padx=10, pady=5)

        # Output directory selector
        ttk.Label(output_frame, text="Output Directory:").grid(row=0, column=0, sticky=tk.W, padx=5, pady=5)
        ttk.Entry(output_frame, textvariable=self.output_dir, width=60).grid(row=0, column=1, padx=5, pady=5)
        ttk.Button(output_frame, text="Browse...", command=self.browse_output_dir).grid(row=0, column=2, padx=5, pady=5)

    def create_log_section(self, parent):
        log_frame = ttk.LabelFrame(parent, text="Processing Log", padding="10")
        log_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=5)

        # Log text area
        self.log_text = scrolledtext.ScrolledText(log_frame, wrap=tk.WORD, width=80, height=10)
        self.log_text.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)
        self.log_text.config(state=tk.DISABLED)

    def create_control_buttons(self, parent):
        button_frame = ttk.Frame(parent)
        button_frame.pack(fill=tk.X, padx=10, pady=5)

        # Control buttons
        self.start_button = ttk.Button(button_frame, text="Start Processing", command=self.start_processing)
        self.start_button.pack(side=tk.LEFT, padx=5)

        self.stop_button = ttk.Button(button_frame, text="Stop", command=self.stop_processing, state=tk.DISABLED)
        self.stop_button.pack(side=tk.LEFT, padx=5)

        ttk.Button(button_frame, text="Exit", command=self.root.destroy).pack(side=tk.RIGHT, padx=5)

    def browse_comments_file(self):
        filepath = filedialog.askopenfilename(
            filetypes=[("CSV files", "*.csv"), ("Excel files", "*.xlsx;*.xls"), ("All files", "*.*")]
        )
        if filepath:
            self.comments_file_path.set(filepath)

    def browse_videos_file(self):
        filepath = filedialog.askopenfilename(
            filetypes=[("Excel files", "*.xlsx;*.xls"), ("CSV files", "*.csv"), ("All files", "*.*")]
        )
        if filepath:
            self.videos_file_path.set(filepath)

    def browse_output_dir(self):
        dirpath = filedialog.askdirectory()
        if dirpath:
            self.output_dir.set(dirpath)

    def log(self, message):
        self.root.after(0, self._log, message)

    def _log(self, message):
        self.log_text.config(state=tk.NORMAL)
        self.log_text.insert(tk.END, f"{message}\n")
        self.log_text.see(tk.END)
        self.log_text.config(state=tk.DISABLED)

    def update_status(self, message):
        self.status_var.set(message)

    def update_progress(self, value):
        self.progress_var.set(value)

    def start_processing(self):
        if not self.comments_file_path.get() and not self.videos_file_path.get():
            messagebox.showerror("Error", "Please select at least one input file.")
            return

        self.start_button.config(state=tk.DISABLED)
        self.stop_button.config(state=tk.NORMAL)
        self.stop_requested = False

        self.log_text.config(state=tk.NORMAL)
        self.log_text.delete(1.0, tk.END)
        self.log_text.config(state=tk.DISABLED)

        self.processing_thread = threading.Thread(target=self.process_files)
        self.processing_thread.daemon = True
        self.processing_thread.start()

    def stop_processing(self):
        self.stop_requested = True
        self.log("Stop requested. Waiting for current batch to complete...")
        self.update_status("Stopping...")

    def process_files(self):
        try:
            self.log("Starting keyword extraction process...")
            self.update_status("Initializing...")

            comment_column = self.comment_column_var.get()
            transcription_column = self.transcription_column_var.get()
            top_n = self.top_n_var.get()
            batch_size = self.batch_size_var.get()
            output_dir = self.output_dir.get()

            # --- Text cleaning and processing functions ---
            self.log("Setting up text processing functions...")

            # Get all emoji characters known by the emoji library
            emoji_chars = "".join(emoji.EMOJI_DATA.keys())
            # Create a regex pattern to keep word characters, whitespace, AND emojis
            # This pattern excludes characters that are NOT word chars, whitespace, or in the emoji set
            keep_pattern = f"[^\w\s{re.escape(emoji_chars)}]"

            def clean_text(text):
                if not isinstance(text, str):
                    return ""
                text = text.lower()
                # Remove URLs first
                text = re.sub(r'http\S+', '', text)
                # Remove digits
                text = re.sub(r'\d+', '', text)
                # Remove punctuation EXCEPT emojis
                text = re.sub(keep_pattern, '', text, flags=re.UNICODE)
                # Remove extra whitespace
                text = re.sub(r'\s+', ' ', text).strip()
                return text

            stop_words = set(stopwords.words('english'))
            lemmatizer = WordNetLemmatizer()

            def process_text(text):
                # Tokenize (word_tokenize might need refinement for complex emoji handling if issues arise)
                tokens = word_tokenize(text)
                # Remove stopwords (emojis are unlikely to be stopwords)
                tokens_no_stopwords = [word for word in tokens if word not in stop_words]
                # Lemmatize (lemmatizer should ignore emojis)
                lemmatized = [lemmatizer.lemmatize(token) for token in tokens_no_stopwords]
                return ' '.join(lemmatized)

            # --- Load data (Ensure error handling for file loading) ---
            comments_df = None
            videos_df = None

            try:
                if self.comments_file_path.get():
                    self.log(f"Loading comments data from {self.comments_file_path.get()}...")
                    self.update_status("Loading comments data...")
                    if self.comments_file_path.get().lower().endswith('.csv'):
                        comments_df = pd.read_csv(self.comments_file_path.get())
                    else:
                        comments_df = pd.read_excel(self.comments_file_path.get())
                    self.log(f"Successfully loaded comments data with {len(comments_df)} rows")
                    self.log(f"Columns: {comments_df.columns.tolist()}")
                    # Validate comment column
                    if comment_column not in comments_df.columns:
                         self.log(f"Warning: '{comment_column}' not found. Available: {comments_df.columns.tolist()}. Attempting fallback...")
                         # Add fallback logic or error out
                         # For now, let's assume it fails if not found after potential fallback
                         potential_columns = [col for col in comments_df.columns if 'comment' in col.lower()]
                         if potential_columns:
                             comment_column = potential_columns[0]
                             self.log(f"Using '{comment_column}' instead.")
                         else:
                             raise ValueError(f"Specified comment column '{self.comment_column_var.get()}' not found.")


            except Exception as e:
                self.log(f"Error loading comments file: {e}")
                messagebox.showerror("File Load Error", f"Failed to load comments file: {e}")
                comments_df = None # Ensure it's None if loading fails

            try:
                if self.videos_file_path.get():
                    self.log(f"Loading videos data from {self.videos_file_path.get()}...")
                    self.update_status("Loading videos data...")
                    if self.videos_file_path.get().lower().endswith('.csv'):
                        videos_df = pd.read_csv(self.videos_file_path.get())
                    else:
                        videos_df = pd.read_excel(self.videos_file_path.get())
                    self.log(f"Successfully loaded videos data with {len(videos_df)} rows")
                    self.log(f"Columns: {videos_df.columns.tolist()}")
                     # Validate transcription column
                    if transcription_column not in videos_df.columns:
                        self.log(f"Warning: '{transcription_column}' not found. Available: {videos_df.columns.tolist()}. Attempting fallback...")
                        # Add fallback logic or error out
                        potential_columns = [col for col in videos_df.columns if 'transcript' in col.lower() or 'text' in col.lower()]
                        if potential_columns:
                            transcription_column = potential_columns[0]
                            self.log(f"Using '{transcription_column}' instead.")
                        else:
                            raise ValueError(f"Specified transcription column '{self.transcription_column_var.get()}' not found.")

            except Exception as e:
                self.log(f"Error loading videos file: {e}")
                messagebox.showerror("File Load Error", f"Failed to load videos file: {e}")
                videos_df = None # Ensure it's None if loading fails


            # --- Initialize KeyBERT ---
            self.log("Initializing KeyBERT model...")
            self.update_status("Initializing KeyBERT model...")
            kw_model = KeyBERT()

            # --- Function to extract keywords ---
            def extract_keywords(text, top_n_kw=5):
                if not text or len(text.split()) < 3: # Check if text is too short
                    return []
                try:
                    # Pass the *cleaned but not processed* text for better context preservation if needed
                    # Or pass processed_text if lemmatization/stopword removal is desired *before* keyword extraction
                    keywords = kw_model.extract_keywords(
                        text, # Using the processed_text here
                        keyphrase_ngram_range=(1, 2),
                        stop_words='english', # KeyBERT can handle its own stopwords
                        use_mmr=True,
                        diversity=0.7,
                        top_n=top_n_kw
                    )
                    # Return list of (keyword, score) tuples
                    return keywords if keywords else []
                except Exception as e:
                    self.log(f"Error extracting keywords for text chunk: {e}")
                    # self.log(f"Problematic text (first 100 chars): {text[:100]}") # Optional: log problematic text
                    return [] # Return empty list on error for this chunk

            # --- Process comments if available ---
            if comments_df is not None:
                self.log("\nProcessing comments data...")
                self.update_status("Processing comments data...")

                # Clean and preprocess text
                self.log("Cleaning comments...")
                comments_df['clean_text'] = comments_df[comment_column].apply(clean_text)
                self.log("Preprocessing comments (tokenize, stopwords, lemmatize)...")
                comments_df['processed_text'] = comments_df['clean_text'].apply(process_text)


                # Extract keywords in batches
                self.log("Extracting keywords from comments...")
                keywords_list_comments = []
                total_comments = len(comments_df)

                for i in range(0, total_comments, batch_size):
                    if self.stop_requested:
                        self.log("Comment processing stopped by user.")
                        break

                    batch_end = min(i + batch_size, total_comments)
                    current_batch_num = (i // batch_size) + 1
                    total_batches = (total_comments + batch_size - 1) // batch_size

                    self.log(f"Processing comment batch {current_batch_num}/{total_batches} (rows {i+1} to {batch_end})")
                    self.update_status(f"Processing comments: {batch_end}/{total_comments}")

                    # Use 'processed_text' for keyword extraction
                    batch_texts = comments_df['processed_text'].iloc[i:batch_end]
                    batch_keywords = [extract_keywords(text, top_n) for text in batch_texts]
                    keywords_list_comments.extend(batch_keywords)

                    progress = (batch_end / total_comments) * 50 # Comments take first 50%
                    self.update_progress(progress)

                if not self.stop_requested:
                    comments_df['keywords'] = keywords_list_comments
                    comments_df['keywords_only'] = comments_df['keywords'].apply(
                        lambda kw_list: [k for k, _ in kw_list] if isinstance(kw_list, list) else []
                    )

                    output_path = os.path.join(output_dir, 'comments_with_keywords.csv')
                    try:
                        self.log(f"Saving processed comments to {output_path}")
                        comments_df.to_csv(output_path, index=False, encoding='utf-8-sig') # Ensure UTF-8 for emojis
                        self.log("Saved comments successfully.")
                        self.log("\nSample keywords from comments:")
                        for idx in range(min(3, len(comments_df))):
                             original_comment = comments_df[comment_column].iloc[idx]
                             extracted_kws = comments_df['keywords'].iloc[idx]
                             self.log(f"Comment (Original): {str(original_comment)[:100]}...")
                             self.log(f"Keywords: {extracted_kws}")
                             self.log("-" * 20)
                    except Exception as e:
                         self.log(f"Error saving comments file: {e}")
                         messagebox.showerror("File Save Error", f"Failed to save comments file: {e}")


            # --- Process videos if available ---
            if videos_df is not None and not self.stop_requested:
                self.log("\nProcessing video transcription data...")
                self.update_status("Processing video transcriptions...")

                self.log("Cleaning transcriptions...")
                videos_df['clean_text'] = videos_df[transcription_column].apply(clean_text)
                self.log("Preprocessing transcriptions (tokenize, stopwords, lemmatize)...")
                videos_df['processed_text'] = videos_df['clean_text'].apply(process_text)

                self.log("Extracting keywords from transcriptions...")
                keywords_list_videos = []
                total_videos = len(videos_df)

                for i in range(0, total_videos, batch_size):
                     if self.stop_requested:
                         self.log("Video processing stopped by user.")
                         break

                     batch_end = min(i + batch_size, total_videos)
                     current_batch_num = (i // batch_size) + 1
                     total_batches = (total_videos + batch_size - 1) // batch_size

                     self.log(f"Processing transcription batch {current_batch_num}/{total_batches} (rows {i+1} to {batch_end})")
                     self.update_status(f"Processing transcriptions: {batch_end}/{total_videos}")

                     batch_texts = videos_df['processed_text'].iloc[i:batch_end]
                     batch_keywords = [extract_keywords(text, top_n) for text in batch_texts]
                     keywords_list_videos.extend(batch_keywords)

                     # Update progress (videos take second 50% or full 100% if no comments)
                     base_progress = 50 if comments_df is not None else 0
                     progress = base_progress + (batch_end / total_videos) * (100 - base_progress)
                     self.update_progress(progress)


                if not self.stop_requested:
                    videos_df['keywords'] = keywords_list_videos
                    videos_df['keywords_only'] = videos_df['keywords'].apply(
                        lambda kw_list: [k for k, _ in kw_list] if isinstance(kw_list, list) else []
                    )

                    output_path = os.path.join(output_dir, 'videos_with_keywords.csv')
                    try:
                        self.log(f"Saving processed videos to {output_path}")
                        videos_df.to_csv(output_path, index=False, encoding='utf-8-sig') # Ensure UTF-8 for emojis
                        self.log("Saved videos successfully.")
                        self.log("\nSample keywords from videos:")
                        for idx in range(min(3, len(videos_df))):
                             original_transcript = videos_df[transcription_column].iloc[idx]
                             extracted_kws = videos_df['keywords'].iloc[idx]
                             self.log(f"Transcription (Original): {str(original_transcript)[:100]}...")
                             self.log(f"Keywords: {extracted_kws}")
                             self.log("-" * 20)
                    except Exception as e:
                         self.log(f"Error saving videos file: {e}")
                         messagebox.showerror("File Save Error", f"Failed to save videos file: {e}")


            # --- Completion message ---
            if self.stop_requested:
                self.update_status("Processing stopped by user")
                self.log("Processing stopped before completion.")
            else:
                self.update_status("Processing completed")
                self.log("Keyword extraction completed successfully!")
                self.update_progress(100)
                messagebox.showinfo("Success", "Keyword extraction completed successfully!")

        except FileNotFoundError as e:
             self.log(f"Error: Input file not found - {e}")
             self.update_status("Error: File not found")
             messagebox.showerror("File Not Found", f"Could not find the specified file: {e}")
        except ValueError as e: # Catch column errors
             self.log(f"Configuration Error: {e}")
             self.update_status("Error: Column not found")
             messagebox.showerror("Configuration Error", str(e))
        except Exception as e:
            self.log(f"An unexpected error occurred during processing: {e}")
            self.log(traceback.format_exc()) # Log the full traceback for debugging
            self.update_status("Error occurred")
            messagebox.showerror("Error", f"An unexpected error occurred: {str(e)}")

        finally:
            # Ensure buttons are reset in the main thread
            self.root.after(0, lambda: self.start_button.config(state=tk.NORMAL))
            self.root.after(0, lambda: self.stop_button.config(state=tk.DISABLED))


# Main application launcher
if __name__ == "__main__":
    # Check for required packages before starting GUI
    try:
        import pandas as pd
        import numpy as np
        import nltk
        from keybert import KeyBERT
        import emoji # Check for emoji library too
        print("Required packages found.")
    except ImportError as e:
        print(f"Error: Missing required package: {e}")
        print("Please install the required packages. You might need:")
        print("pip install pandas numpy nltk keybert transformers torch scikit-learn openpyxl emoji")
        # Show error in a simple Tkinter window if possible, as main GUI might not start
        try:
            root_check = tk.Tk()
            root_check.withdraw() # Hide the main window
            messagebox.showerror("Missing Packages",
                                 f"Missing required package: {e.name}.\n\nPlease install packages, e.g.,:\n"
                                 "pip install pandas numpy nltk keybert transformers torch scikit-learn openpyxl emoji")
            root_check.destroy()
        except tk.TclError:
             # Handle cases where Tkinter itself might be missing or has issues
            print("Tkinter error, cannot display GUI message box.")
        sys.exit(1) # Exit if packages are missing


    # Pre-download NLTK (optional but good practice) - simplified check
    # print("Checking NLTK resources...")
    # resources = ['punkt', 'stopwords', 'wordnet', 'omw-1.4']
    # for resource in resources:
    #     try:
    #         nltk.data.find(f'corpora/{resource}' if resource != 'punkt' else f'tokenizers/{resource}')
    #     except LookupError:
    #         print(f"Downloading NLTK resource: {resource}...")
    #         nltk.download(resource, quiet=True) # Download quietly

    # Create and run the application
    root = tk.Tk()
    app = KeywordExtractionApp(root)
    root.mainloop()

<>:230: SyntaxWarning: invalid escape sequence '\w'
<>:230: SyntaxWarning: invalid escape sequence '\w'
C:\Users\User\AppData\Local\Temp\ipykernel_4816\2717259807.py:230: SyntaxWarning: invalid escape sequence '\w'
  keep_pattern = f"[^\w\s{re.escape(emoji_chars)}]"


Required packages found.


In [19]:
#STEPS 11 - Implementing BERT model
#Try Accuracy Testing MentalBERT

# Install dependencies (uncomment in Colab or terminal)
# !pip install transformers torch pandas scikit-learn

!pip install transformers
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
from sklearn.preprocessing import normalize

# Your Hugging Face token
hf_token = "hf_..."  # Replace with your actual token

# Load MentalBERT model and tokenizer using the token
try:
    tokenizer = AutoTokenizer.from_pretrained("mental/mental-bert-base-uncased", token=hf_token)
    model = AutoModel.from_pretrained("mental/mental-bert-base-uncased", token=hf_token)
except TypeError:
    print("Token argument not accepted or login required. Ensure you are logged in via CLI or notebook_login().")
    tokenizer = AutoTokenizer.from_pretrained("mental/mental-bert-base-uncased")
    model = AutoModel.from_pretrained("mental/mental-bert-base-uncased")

# Function to generate embedding using MentalBERT
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze()

# Function to generate embeddings for a batch of texts
def get_batch_embeddings(texts):
    # Ensure all texts are strings before tokenizing
    texts = [str(text) for text in texts]  # Convert all to strings
    inputs = tokenizer(texts, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)

# Function to rank keywords against a user input with batch processing
def rank_keywords_batch(df, keyword_column, user_input, batch_size=10):
    user_embedding = get_embedding(user_input)
    rankings = []

    # Process keywords in batches with tqdm progress bar
    for i in tqdm(range(0, len(df[keyword_column]), batch_size), desc="Processing keywords"):
        # Get the current batch of keywords
        keywords_batch = df[keyword_column].dropna().unique()[i:i+batch_size]

        # Ensure the batch is not empty
        if not keywords_batch.size: # Changed this line
            continue

        # Convert all keywords to string before embedding
        keywords_batch = [str(keyword) for keyword in keywords_batch]

        try:
            keyword_embeddings = get_batch_embeddings(keywords_batch)

            # Move embeddings to CPU if necessary before calculating similarity
            keyword_embeddings = keyword_embeddings.cpu().numpy()
            normalized_user_embedding = normalize(user_embedding.cpu().numpy().reshape(1, -1)) #Added normalization and reshape
            normalized_keyword_embeddings = normalize(keyword_embeddings)
            similarities = cosine_similarity(normalized_user_embedding, normalized_keyword_embeddings)

            # Rank keywords based on similarity
            for keyword, score in zip(keywords_batch, similarities[0]): # changed similarities to similarities[0]
                rankings.append((keyword, score))

        except Exception as e:
            print(f"Error processing batch of keywords: {str(e)}")
            continue

    return sorted(rankings, key=lambda x: x[1], reverse=True)

# Load your CSV files (make sure paths are correct)
videos_df = pd.read_csv("G:/Other computers/My Laptop (1)/Pelajaran UiTM/Classes and Lectures (Semester 6)/CSP650/Developments/Simplified Data/Testings/comments_with_keywords.csv")
comments_df = pd.read_csv("G:/Other computers/My Laptop (1)/Pelajaran UiTM/Classes and Lectures (Semester 6)/CSP650/Developments/Simplified Data/Testings/videos_with_keywords.csv")

# Clean up the data (remove any rows with missing 'keywords')
videos_df = videos_df.dropna(subset=["keywords"])
comments_df = comments_df.dropna(subset=["keywords"])

# Input from user
user_input = "I'm feeling down"

# Rank keywords separately for videos and comments using batch processing
video_ranks = rank_keywords_batch(videos_df, "keywords", user_input)
comment_ranks = rank_keywords_batch(comments_df, "keywords", user_input)

# Convert results to DataFrame and display
video_results = pd.DataFrame(video_ranks, columns=["Keyword", "Similarity Score"])
comment_results = pd.DataFrame(comment_ranks, columns=["Keyword", "Similarity Score"])

print("\nTop Video Keywords Relevant to Input:")
print(video_results.head(10))

print("\nTop Comment Keywords Relevant to Input:")
print(comment_results.head(10))


Some weights of BertModel were not initialized from the model checkpoint at mental/mental-bert-base-uncased and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Processing keywords: 100%|███████████████████████████████████████████████████████████████| 6/6 [00:01<00:00,  3.46it/s]


Top Video Keywords Relevant to Input:
                                             Keyword  Similarity Score
0  [('want antidepressant', 0.6271), ('feeling aw...          0.667951
1  [('drs scared', 0.7271), ('im scared', 0.5479)...          0.653591
2  [('diagnosed anxiety', 0.6303), ('im heartbrok...          0.651593
3  [('scared feel', 0.5524), ('im starting', 0.43...          0.651471
4  [('im lonely', 0.6674), ('feeling everyday', 0...          0.649974
5  [('peace depression', 0.7632), ('help peace', ...          0.649039
6  [('depression anxiety', 0.5873), ('dad like', ...          0.648247
7  [('zoloft anxiety', 0.6363), ('diagnosed adhd'...          0.648160
8  [('depression disability', 0.6849), ('tired li...          0.647352
9  [('sayimgi understand', 0.6682), ('depression ...          0.647301

Top Comment Keywords Relevant to Input:
                                             Keyword  Similarity Score
0  [('anxiety feel', 0.6967), ('lightheadedness d...          0.6375